# Loading n-PDFs


In [111]:
from pyalex import Works
import sys
import os

sys.path.append("../code")
from util import list_to_text_file, get_string_from_text_file

works = (
    Works()
    .filter(is_oa=True)
    .select(["open_access", "display_name"])
    .get(per_page=100, page=1)
)

In [112]:
import requests
import os

TIMEOUT = 15


# move to util?
def extract_url_to_pdf(data_dir, url: str, filename: str) -> bool:
    try:
        print(f"trying to get pdf from {url}...")
        response = requests.get(url, timeout=TIMEOUT)
        if response.status_code != 200:
            raise Exception("Request GET error. status code not 200")

        PDF_DIR = os.path.join(data_dir, "PDF")
        output_path = os.path.join(PDF_DIR, filename + ".pdf")

        with open(output_path, "wb") as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"Could not extract pdf from url. Error: {e}")
        return False

In [113]:
def parse_n_papers_to_pdf_file(num_papers: int):
    DATA_DIR = "../unilex-transcript/data"
    output_list = get_string_from_text_file(
        os.path.join(DATA_DIR, "extraction_paper_list.txt"), full_path=True
    ).splitlines()
    exception_list = get_string_from_text_file(
        os.path.join(DATA_DIR, "extraction_paper_exception_list.txt"), full_path=True
    ).splitlines()
    output_list = [paper for paper in output_list if paper not in exception_list]
    output_cnt = len(output_list)

    added_cnt = 0
    page_idx = 0
    while output_cnt < num_papers:
        page_idx += 1
        works = (
            Works()
            .filter(is_oa=True)
            .select(["open_access", "display_name"])
            .get(per_page=100, page=page_idx)
        )
        for work in works:
            try:
                url = work["open_access"]["oa_url"]
                name = work["display_name"]
                name = name.replace(" ", "_")
                if name in exception_list:
                    continue
                if name is None:
                    raise Exception("name is NoneType")
                if name not in output_list:
                    print("")
                    if not extract_url_to_pdf(DATA_DIR, url, filename=name):
                        raise Exception("could not extract pdf from url")
                    output_list.append(name)
                    output_cnt += 1
                    added_cnt += 1
                    print(f"\n====={output_cnt} pdfs in output list=====\n")
                if output_cnt >= num_papers:
                    break
            except Exception as e:
                if name:
                    exception_list.append(name)
                print("Error msg:")
                print(e)

        list_to_text_file(
            exception_list,
            os.path.join(DATA_DIR, "extraction_paper_exception_list.txt"),
        )
        list_to_text_file(
            output_list,
            os.path.join(DATA_DIR, "extraction_paper_list.txt"),
        )

    print(f"##### FINISHED. {added_cnt} papers added to list #####")

In [114]:
parse_n_papers_to_pdf_file(200)

Error msg:
'NoneType' object has no attribute 'replace'

trying to get pdf from http://www.public.asu.edu/%7Edavidpm/classes/psy536/Baron.pdf...

=====11 pdfs in output list=====


trying to get pdf from https://europepmc.org/articles/pmc450244?pdf=render...

=====12 pdfs in output list=====


trying to get pdf from http://link.aps.org/pdf/10.1103/PhysRev.140.A1133...

=====13 pdfs in output list=====


trying to get pdf from http://www.cell.com/article/S0092867411001279/pdf...
Could not extract pdf from url. Error: Request GET error. status code not 200
Error msg:
could not extract pdf from url

trying to get pdf from https://onlinelibrary.wiley.com/doi/pdfdirect/10.3322/caac.21660...
Could not extract pdf from url. Error: Request GET error. status code not 200
Error msg:
could not extract pdf from url

trying to get pdf from https://www.jstatsoft.org/index.php/jss/article/view/v067i01/v67i01.pdf...
Could not extract pdf from url. Error: [Errno 2] No such file or directory: '../unilex

# Parse PDF to structured HTML


In [115]:
import subprocess

In [116]:
# subprocess.Popen("./pdf2html.py", cwd="/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript")
# doesn't work (input device is not a TTY?). execute pdf2html.py directly
subprocess.run(
    "./pdf2html.py", cwd="/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript"
)


docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/DNA_sequencing_with_chain-terminating_inhibitors /pdf/PDF/DNA_sequencing_with_chain-terminating_inhibitors.pdf DNA_sequencing_with_chain-terminating_inhibitors.html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/THE_ADSORPTION_OF_GASES_ON_PLANE_SURFACES_OF_GLASS,_MICA_AND_PLATINUM. /pdf/PDF/THE_ADSORPTION_OF_GASES_ON_PLANE_SURFACES_OF_GLASS,_MICA_AND_PLATINUM..pdf THE_ADSORPTION_OF_GASES_ON_PLANE_SURFACES_OF_GLASS,_MICA_AND_PLATINUM..html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/p

the input device is not a TTY
the input device is not a TTY
the input device is not a TTY
the input device is not a TTY




docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/Scoping_studies:_towards_a_methodological_framework /pdf/PDF/Scoping_studies:_towards_a_methodological_framework.pdf Scoping_studies:_towards_a_methodological_framework.html
docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/Highly_accurate_protein_structure_prediction_with_AlphaFold /pdf/PDF/Highly_accurate_protein_structure_prediction_with_AlphaFold.pdf Highly_accurate_protein_structure_prediction_with_AlphaFold.html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-mas

sh: -c: line 0: syntax error near unexpected token `('
sh: -c: line 0: `docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/PRISMA_Extension_for_Scoping_Reviews_(PRISMA-ScR):_Checklist_and_Explanation /pdf/PDF/PRISMA_Extension_for_Scoping_Reviews_(PRISMA-ScR):_Checklist_and_Explanation.pdf PRISMA_Extension_for_Scoping_Reviews_(PRISMA-ScR):_Checklist_and_Explanation.html'
the input device is not a TTY
the input device is not a TTY
the input device is not a TTY



docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/GRADE:_an_emerging_consensus_on_rating_quality_of_evidence_and_strength_of_recommendations /pdf/PDF/GRADE:_an_emerging_consensus_on_rating_quality_of_evidence_and_strength_of_recommendations.pdf GRADE:_an_emerging_consensus_on_rating_quality_of_evidence_and_strength_of_recommendations.html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/Preferred_Reporting_Items_for_Systematic_Reviews_and_Meta-Analyses:_The_PRISMA_Statement /pdf/PDF/Preferred_Reporting_Items_for_Systematic_Reviews_and_Meta-Analyses:_The_PRISMA_Statement.pdf Preferred_Reporting_Item

...


docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/Note_on_an_Approximation_Treatment_for_Many-Electron_Systems /pdf/PDF/Note_on_an_Approximation_Treatment_for_Many-Electron_Systems.pdf Note_on_an_Approximation_Treatment_for_Many-Electron_Systems.html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/Normalized_cuts_and_image_segmentation /pdf/PDF/Normalized_cuts_and_image_segmentation.pdf Normalized_cuts_and_image_segmentation.html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --e

the input device is not a TTY
the input device is not a TTY
the input device is not a TTY



docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/Inhibited_Spontaneous_Emission_in_Solid-State_Physics_and_Electronics /pdf/PDF/Inhibited_Spontaneous_Emission_in_Solid-State_Physics_and_Electronics.pdf Inhibited_Spontaneous_Emission_in_Solid-State_Physics_and_Electronics.html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20200820-ubuntu-20.04-x86_64 --embed-external-font 0  --process-nontext 0 --embed cfijo --dest-dir /pdf/HTML/Emergence_of_Scaling_in_Random_Networks /pdf/PDF/Emergence_of_Scaling_in_Random_Networks.pdf Emergence_of_Scaling_in_Random_Networks.html

docker run -ti --rm -v /Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data:/pdf -w /pdf pdf2htmlex/pdf2htmlex:0.18.8.rc2-master-20

the input device is not a TTY
the input device is not a TTY
the input device is not a TTY


CompletedProcess(args='./pdf2html.py', returncode=0)

In [117]:
subprocess.run(
    "./transcript.py", cwd="/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript"
)

Loading fonts from: /path/to/truetype/fonts/
/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data/HTML/A_Simple,_Positive_Semi-Definite,_Heteroskedasticity_and_Autocorrelation_Consistent_Covariance_Matrix/A_Simple,_Positive_Semi-Definite,_Heteroskedasticity_and_Autocorrelation_Consistent_Covariance_Matrix.html
list index out of range
Traceback (most recent call last):
  File "/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/./transcript.py", line 427, in batch_process
    semanticize(path)
  File "/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/./transcript.py", line 319, in semanticize
    h_levels = heading_levels(dom, dimensions)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/./transcript.py", line 276, in heading_levels
    prevalent_fs = top_stats[0][-1]
                   ~~~~~~~~~^^^
IndexError: list index out of range

/Users/ms/cs/ML/MLab/paper-extraction/unilex-transcript/data/HTML/SSD:_Sin

CompletedProcess(args='./transcript.py', returncode=0)

# Parse HTML with python


# -


# -


In [109]:
Works().filter(is_oa=True).count()

53858599